In [19]:
#import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")

In [20]:
using JuMP, GLPK
using DataFrames
using CSV

In [21]:
#load cost data
con_generation = CSV.File("../data/conventional_generators.csv") |> DataFrame

#load wind_technicaldata
wind_generation = CSV.File("../data/wind_farms.csv") |> DataFrame

# load wind profile
wind_profile = CSV.File("../data/wind_powerprofile_200.csv") |> DataFrame

# batteries
battery = CSV.File("../data/Battery.csv") |> DataFrame

# transmission_lines
transmission_lines = CSV.File("../data/transmission_lines.csv") |> DataFrame

# transmission_lines_zone
transmission_lines_zone = CSV.File("../data/transmission_lines_zone.csv") |> DataFrame

# node to zone 
node_to_zone = CSV.File("../data/node_to_zone.csv") |> DataFrame


Row,Node,Zone
,Int64,Int64
1,1,1
2,2,1
3,3,1
4,4,1
5,5,1
6,6,1
7,7,1
8,8,1
9,9,1


In [22]:
#demand bids
name = "demand_hour_0.csv"
demand_bids = CSV.File("../data/demand_bids_hour/" * name) |> DataFrame
 
# number of convential generators
G = size(con_generation, 1)

# number of demand
D = size(demand_bids, 1)

# number of wind generators
W = size(wind_generation, 1)

# number of batteries
B = size(battery, 1)

# number of transmission lines
L = size(transmission_lines, 1)

# number of nodes
N = 24

# number of zones
Z = size(transmission_lines_zone, 1)


3

In [23]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[])

# For x variables, manually add each column. This is a one-time setup.
for i in 1:G
    result_df[!, Symbol("x_con$i")] = Float64[]
end

# For w variables, manually add each column. This is a one-time setup.
for i in 1:W
    result_df[!, Symbol("x_wind$i")] = Float64[]
end

# For y variables, manually add each column. This is a one-time setup.
for i in 1:D
    result_df[!, Symbol("y$i")] = Float64[]
end

# For b variables, manually add each column. This is a one-time setup.
for i in 1:B
    result_df[!, Symbol("x_bat_charg$i")] = Float64[]
    result_df[!, Symbol("x_bat_discharg$i")] = Float64[]
end

# For l variables, manually add each column. This is a one-time setup.
for i in 1:Z
    result_df[!, Symbol("x_line$i")] = Float64[]
end



equilibrium_df = DataFrame(hour = Int[])

# for each zone a market price column
for i in 1:Z
    equilibrium_df[!, Symbol("market_price_zone$i")] = Float64[]
end



In [24]:
println(result_df)

0×47 DataFrame
 Row │ hour   x_con1   x_con2   x_con3   x_con4   x_con5   x_con6   x_con7   x_con8   x_con9   x_con10  x_con11  x_con12  x_wind1  x_wind2  x_wind3  x_wind4  y1       y2       y3       y4       y5       y6       y7       y8       y9       y10      y11      y12      y13      y14      y15      y16      y17      x_bat_charg1  x_bat_discharg1  x_bat_charg2  x_bat_discharg2  x_bat_charg3  x_bat_discharg3  x_bat_charg4  x_bat_discharg4  x_bat_charg5  x_bat_discharg5  x_line1  x_line2  x_line3 
     │ Int64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64       Float64          Float64       Float64          Float64       Float64          Float64       Float64          Float64       Float64          Float64  Float64  Float64 

In [25]:
# Load demand bids data
demand_bids_all = [CSV.File("../data/demand_bids_hour/demand_hour_$(i-1).csv") |> DataFrame for i in 1:24]

# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for every generator and hour
@variable(model, x_con[1:G, 1:24] >= 0)  # Power output variable for conventional generators
@variable(model, x_wind[1:W, 1:24] >= 0)  # Power output variable for wind generators
@variable(model, y[1:D, 1:24] >= 0)  # Demand variable
@variable(model, bat_char[1:B, 1:24])  # Battery variable when it is charging
@variable(model, bat_disch[1:B, 1:24])  # Battery variable when it is discharging
@variable(model, l[1:Z, 1:24])  # Transmission line variable

# Add initial values for the battery
init_bat = []
for b in 1:B
    append!(init_bat, 0)
end

# Define the charging and discharging efficiencies of the battery
eff_char = 0.9
eff_disch= 0.9

# Add constraints for each plant
for g in 1:G
    for hour in 1:24
        @constraint(model, x_con[g, hour] <= con_generation[g, 6])  # Set the upper bound
        @constraint(model, x_con[g, hour] >= 0)  # Set the lower bound
    end
end

# Add constraints for each wind farm
for w in 1:W
    for hour in 1:24
        @constraint(model, x_wind[w, hour] <= wind_profile[hour, w+1])  # Set the upper bound
        @constraint(model, x_wind[w, hour] >= 0)  # Set the lower bound
    end
end

# Add constraints for each demand bid
for d in 1:D
    for hour in 1:24
        @constraint(model, y[d, hour] <= demand_bids_all[hour][d, 2])  # Set the upper bound
        @constraint(model, y[d, hour] >= 0)  # Set the lower bound
    end
end

#add constraints for initial con_generation
for g in 1:G
    @constraint(model, x_con[g, 1] <= con_generation[g, 11] + con_generation[g, 9])  # Set the upper bound
    @constraint(model, x_con[g, 1] >= con_generation[g, 11] - con_generation[g, 10])  # Set the lower bound
end

# Add temporary for each generator
for g in 1:G
    for hour in 2:24
        @constraint(model, x_con[g, hour] <= x_con[g, hour - 1] + con_generation[g, 7])  # Set the upper bound
        @constraint(model, x_con[g, hour] >= x_con[g, hour - 1] - con_generation[g, 8])  # Set the lower bound
    end
end

# add Power constraints for battery charging
for b in 1:B
    for hour in 1:24
        @constraint(model, bat_char[b, hour] <= battery[b, 5])  # Set the upper bound
        @constraint(model, bat_char[b, hour] >= 0)  # Set the lower bound 
    end
end

# add Power constraints for battery discharging, with positive values
for b in 1:B
    for hour in 1:24
        @constraint(model, bat_disch[b, hour] <= battery[b, 5])  # Set the upper bound
        @constraint(model, bat_disch[b, hour] >= 0)  # Set the lower bound
    end
end

# add Capacity constraints for battery charging and discharging
# sum all previous battery charging and discharging, they need to be below the battery capacity
for b in 1:B
    for hour in 1:24
        @constraint(model, sum(init_bat[b] + bat_char[b, h]*eff_char - bat_disch[b, h]/eff_disch for h in 1:hour) <= battery[b, 4])  # Set the upper bound     ##Add initial value + for 1 to 24 instead of 2 to 24
        @constraint(model, sum(init_bat[b] + bat_char[b, h]*eff_char - bat_disch[b, h]/eff_disch for h in 1:hour) >= 0)  # Set the lower bound
    end
end

# add constraints for transmission lines
for line in 1:Z
    for hour in 1:24
        @constraint(model, l[line, hour] <= transmission_lines_zone[line, 4])  # Set the upper bound  
        @constraint(model, l[line, hour] >=  -1 * transmission_lines_zone[line, 4])  # Set the lower bound
    end
end

balance = Vector{Any}(undef, Z*24)

# add balance constraint for each zone and each demand_hour_
for hour in 1:24
    for zone in 1:Z
        balance[(hour-1)*Z+zone] = @constraint(model, sum(x_con[g, hour] for g in 1:G if node_to_zone[con_generation[g, 2],2] == zone) +
                            sum(x_wind[w, hour] for w in 1:W if node_to_zone[wind_generation[w, 2],2] == zone)  +
                            sum(l[line, hour] for line in 1:Z if transmission_lines_zone[line, 2] == zone) -
                            sum(bat_char[b, hour] for b in 1:B if node_to_zone[battery[b, 2], 2] == zone) +
                            sum(bat_disch[b, hour] for b in 1:B if node_to_zone[battery[b, 2], 2] == zone) 
                            ==
                            sum(y[d, hour] for d in 1:D if node_to_zone[demand_bids_all[hour][d, 1],2] == zone) +
                            sum(l[line, hour] for line in 1:Z if transmission_lines_zone[line, 1] == zone) )
    end
end

# Define the objective function
@objective(model, Max, sum(demand_bids_all[hour][d, 3] * y[d, hour] for d in 1:D, hour in 1:24) - sum(con_generation[g, 3] * x_con[g, hour] for g in 1:G, hour in 1:24) - sum(0.0001 * bat_char[b, hour] for hour in 1:24, b in 1:B) - sum(0.0001 * bat_disch[b, hour] for hour in 1:24, b in 1:B) ) # Add the penalty of battery charging and discharging 


# Solve the model
optimize!(model)


# Check the status of the solution
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")

    # RETURN OBJECTIVE value
    println("Objective value: ", objective_value(model))


else
    println("No optimal solution found")
end


Optimal solution found
Objective value: 607836.9614840697


In [26]:
# print decision variables
empty!(result_df)
for hour in 1:24
    resultvector = zeros(1+G+W+D+B+B+Z)
    resultvector[1] = hour
    for g in 1:G
        resultvector[1+g] = value(x_con[g, hour])
    end
    for w in 1:W
        resultvector[1+G+w] = value(x_wind[w, hour])
    end
    for d in 1:D
        resultvector[1+G+W+d] = value(y[d, hour])
    end
    for b in 1:B
        resultvector[1+G+W+D+2*b-1] = value(bat_char[b, hour])
        resultvector[1+G+W+D+2*b] = value(bat_disch[b, hour])
    end
    for line in 1:Z
        resultvector[1+G+W+D+B+line] = value(l[line, hour])
    end
    push!(result_df, resultvector)
end

#save result_df to csv
CSV.write("results/market_clearing_zonal.csv", result_df)

result_df

Row,hour,x_con1,x_con2,x_con3,x_con4,x_con5,x_con6,x_con7,x_con8,x_con9,x_con10,x_con11,x_con12,x_wind1,x_wind2,x_wind3,x_wind4,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,x_bat_charg1,x_bat_discharg1,x_bat_charg2,x_bat_discharg2,x_bat_charg3,x_bat_discharg3,x_bat_charg4,x_bat_discharg4,x_bat_charg5,x_bat_discharg5,x_line1,x_line2,x_line3
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,106.4,106.4,0.0,0.0,0.0,0.0,0.0,229.65,280.0,210.0,217.0,237.475,76.8921,101.54,92.8003,95.3709,67.4817,60.3784,165.153,120.757,197.118,62.1542,207.773,113.653,79.9126,73.7948,46.1717,44.3959,85.2401,78.1367,106.55,108.326,120.757,0.0,0.0,0.0,0.0,10.0,-400.0,0.0,-225.0,5.0,0.0,0.0,0.0,0.0
2,2,106.4,106.4,0.0,0.0,0.0,0.0,0.0,169.65,280.0,210.0,217.0,208.85,66.8277,90.9989,109.167,107.937,63.453,47.5967,155.293,113.547,185.349,58.4435,195.368,106.868,75.1417,105.198,43.4152,41.7454,80.1511,73.4719,100.189,101.859,113.547,0.0,0.0,0.0,0.0,10.0,-400.0,0.0,-225.0,2.59373,0.0,0.0,0.0,0.0
3,3,106.4,65.889,0.0,0.0,0.0,0.0,0.0,109.65,280.0,210.0,217.0,187.381,78.422,116.959,142.88,134.78,60.4314,54.0702,147.898,108.14,176.523,55.6605,186.065,101.779,71.5635,100.189,41.3478,39.7575,76.3344,69.9732,95.418,97.0083,108.14,15.0,0.0,10.0,0.0,10.0,-400.0,0.0,-225.0,4.22827,0.0,0.0,0.0,0.0
4,4,58.4,98.5,0.0,0.0,0.0,0.0,0.0,63.0442,280.0,210.0,217.0,180.225,64.1437,134.249,159.57,133.664,59.4242,53.169,145.433,106.338,173.581,54.7328,182.964,100.083,70.3708,98.5191,40.6587,39.0949,75.0622,68.807,93.8277,95.3915,106.338,15.0,0.0,10.0,0.0,10.0,-400.0,0.0,-225.0,0.0,0.0,0.0,0.0,0.0
5,5,10.4,97.1652,0.0,0.0,0.0,0.0,0.0,39.3472,280.0,210.0,217.0,175.78,102.22,145.507,160.944,165.303,59.4242,53.169,145.433,106.338,173.581,54.7328,182.964,100.083,70.3708,98.5191,40.6587,39.0949,75.0622,68.807,93.8277,95.3915,106.338,15.0,0.0,10.0,0.0,5.55556,-400.0,0.0,-225.0,5.0,0.0,0.0,0.0,0.0
6,6,58.4,58.4,0.0,0.0,0.0,0.0,0.0,99.3472,250.051,210.0,193.1,197.0,134.039,131.112,157.023,161.828,60.4314,54.0702,147.898,108.14,176.523,55.6605,186.065,101.779,71.5635,100.189,41.3478,39.7575,76.3344,69.9732,95.418,97.0083,108.14,15.0,0.0,10.0,0.0,10.0,-385.719,0.0,-225.0,5.0,0.0,0.0,0.0,0.0
7,7,106.4,106.4,0.0,0.0,0.0,0.0,0.0,159.347,280.0,210.0,217.0,245.0,146.517,153.67,141.566,159.927,74.5321,66.6866,182.407,133.373,217.712,68.648,229.48,125.528,88.2616,123.566,50.9956,13.4909,94.1458,86.3003,117.682,119.644,133.373,0.0,0.0,0.0,0.0,0.0,-367.43,0.0,-225.0,0.0,0.0,0.0,0.0,0.0
8,8,106.4,106.4,0.0,0.0,0.0,0.0,0.0,219.347,280.0,210.0,217.0,245.0,143.176,163.392,155.294,170.805,0.0,77.5006,211.987,36.5551,253.017,79.7801,266.693,145.884,102.574,143.604,25.7579,56.9858,109.413,100.295,136.766,139.045,155.001,0.0,15.0,0.0,10.0,0.0,-400.0,0.0,-225.0,0.955778,0.0,0.0,0.0,0.0
9,9,106.4,106.4,0.0,0.0,0.0,0.0,0.0,274.271,280.0,210.0,217.0,245.0,163.297,144.48,172.638,185.318,95.683,0.0,181.251,171.222,279.495,88.1291,294.603,161.15,113.309,158.632,0.0,0.0,120.863,110.791,151.078,153.596,0.0,0.0,0.0,0.0,0.0,0.0,-270.067,0.0,-225.0,5.0,0.0,0.0,0.0,0.0


In [27]:
#save the duals of the balance constraints to the equilibrium_df
empty!(equilibrium_df)
for hour in 1:24
    hour_results = []

    for zone in 1:Z
        push!(hour_results, dual(balance[(hour-1)*Z+zone]))
    end
    push!(equilibrium_df, [hour; hour_results...])
end

#save results to dataframe
CSV.write("results/optimization_results_marketprices_zonal.csv", equilibrium_df)

equilibrium_df

Row,hour,market_price_zone1,market_price_zone2,market_price_zone3
,Int64,Float64,Float64,Float64
1,1,17.6783,10.89,6.02
2,2,17.85,10.89,6.02
3,3,13.32,10.89,6.02
4,4,13.32,10.89,6.02
5,5,13.32,10.89,6.02
6,6,10.52,10.52,5.47
7,7,18.0479,18.0479,6.57
8,8,19.4754,16.4315,6.02
9,9,18.5398,18.5398,6.02
